In [69]:
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [70]:
#read parquet file
#Q1 - Download data and get the length of columns -
df_jn = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_fb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
len(df_jn.columns),len(df_fb.columns)

#ANswer - 19

(19, 19)

In [71]:
#Q2. Computing durationNow let's compute the duration variable. It should contain the duration of a ride in minutes.
#What's the standard deviation of the trips duration in January?

df_jn['duration'] = df_jn.tpep_dropoff_datetime - df_jn.tpep_pickup_datetime
df_jn.duration = df_jn.duration.apply(lambda td: td.total_seconds()/60 )
df_jn.duration.describe()

#Answer std   =   4.259435e+01

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [72]:
len_before_otl = len(df_jn)
len_before_otl

3066766

In [73]:
df_jn = df_jn[(df_jn.duration >= 1) & (df_jn.duration <= 60)]
len_after_otl = len(df_jn)
len_after_otl

3009173

In [74]:
#Q3. Dropping outliers
#Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?
percentage = 100 - (len_before_otl - len_after_otl)/len_before_otl * 100
percentage

Object `outliers` not found.


98.1220282212598

In [75]:
#Q4. One-hot encoding
#Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
#Fit a dictionary vectorizer
#Get a feature matrix from it
#What's the dimensionality of this matrix (number of columns)?

categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
df_jn[categorical] = df_jn[categorical].astype(str)

train_dicts = df_jn[categorical + numerical].to_dict(orient='record')
dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)
x_train.shape[1]


#Answer 516

/tmp/ipykernel_121032/3146462878.py:11: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  train_dicts = df_jn[categorical + numerical].to_dict(orient='record')


516

In [76]:
#Q5. Training a model
#Train a plain linear regression model with default parameters, where duration is the response variable
#Calculate the RMSE of the model on the training data

target = 'duration'
y_train = df_jn[target].values

LR = LinearRegression()
LR.fit(x_train,y_train)
y_pred = LR.predict(x_train)
mean_squared_error(y_train,y_pred, squared=False)

#Answer  7.649134295950567

7.649134295950567

In [77]:
# Evaluation in Feb data 
df_fb['duration'] = df_fb.tpep_dropoff_datetime - df_fb.tpep_pickup_datetime
df_fb.duration = df_fb.duration.apply(lambda td: td.total_seconds()/60 )
df_fb = df_fb[(df_fb.duration >= 1) & (df_fb.duration <= 60)]

categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
df_fb[categorical] = df_fb[categorical].astype(str)

eval_dicts = df_fb[categorical + numerical].to_dict(orient='record')
dv = DictVectorizer()
x_eval = dv.fit_transform(eval_dicts)

target = 'duration'
y_eval = df_fb[target].values

y_pred = LR.predict(x_eval)
mean_squared_error(y_eval,y_pred, squared=False)


/tmp/ipykernel_121032/3386213616.py:10: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  eval_dicts = df_fb[categorical + numerical].to_dict(orient='record')


ValueError: X has 515 features, but LinearRegression is expecting 516 features as input.